In [1]:
# http://qwone.com/~jason/20Newsgroups/

In [2]:
#1. Prosze powtorzyc cwiczenia dla innego podzbioru kategorii

In [3]:
#wczytanie zbioru trenujacego danych tekstowych
from sklearn.datasets import fetch_20newsgroups
categories = [
    'sci.crypt', 
    'rec.sport.baseball', 
    'rec.motorcycles', 
    'sci.space']
train_dataset = fetch_20newsgroups(
    subset='train', 
    categories=categories, 
    shuffle=True, 
    random_state=42
)
test_dataset = fetch_20newsgroups(
    subset='test', 
    categories=categories, 
    shuffle=True, 
    random_state=42
)

In [4]:
#klasy
print(train_dataset.target_names)
print(test_dataset.target_names)

['rec.motorcycles', 'rec.sport.baseball', 'sci.crypt', 'sci.space']
['rec.motorcycles', 'rec.sport.baseball', 'sci.crypt', 'sci.space']


In [5]:
#liczba wiadomosci
print(len(train_dataset.data))
print(len(test_dataset.data))

2383
1585


In [6]:
#wyswietlenie wiadomosci ze zbioru trenujacego o indeksie 180
print(train_dataset.data[180])

From: ajjb@adam4.bnsc.rl.ac.uk (Andrew Broderick)
Subject: Re: Solar Sail Data
Keywords: Solar Sail
Organization: Rutherford Appleton Lab, UK
Lines: 79

In article <1993Apr15.051746.29848@news.duc.auburn.edu> snydefj@eng.auburn.edu writes:
>
>I am looking for any information concerning projects involving Solar
> Sails

I was at an interesting seminar at work (UK's R.A.L. Space Science
Dept.) on this subject, specifically on a small-scale Solar Sail
proposed as a student space project. The guy giving the talk was keen to
generate interest in the project. I'll typein the handout he gave out at
the meeting. Here goes : 

			The Microlight Solar Sail
			-------------------------

1. Introduction
The solar sail is a well-established concept. Harnessing the pressure of
sunlight, a spacecraft would have unlimited range. In principle, such a
vehicle could explore the whole Solar System with zero fuel consumption.

However it is more difficult to design a practical solar sail than most
people r

In [7]:
#wyswietlenie klasy wiadomosci o indeksie 180
print(train_dataset.target_names[train_dataset.target[180]])

sci.space


In [8]:
#prosze wyswietlic wiadomosc o indeksie 5 i jej klase ze zbioru trenujacego
#dlaczego taka wiadomosc ma zadana kategorie?

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='word', ngram_range=(1, 1))
X_train_counts = count_vect.fit_transform(train_dataset.data)

In [10]:
word_list = count_vect.get_feature_names_out()
count_list = X_train_counts.toarray().sum(axis=0)
word_count = dict(zip(word_list,count_list))
sorted(word_count.items(), key=lambda x: x[1], reverse=True)

[('the', 30530),
 ('to', 15205),
 ('of', 13501),
 ('and', 11909),
 ('in', 9861),
 ('is', 8380),
 ('that', 6952),
 ('it', 6349),
 ('for', 5829),
 ('on', 4651),
 ('from', 4554),
 ('you', 4510),
 ('edu', 4471),
 ('be', 4399),
 ('this', 4193),
 ('are', 3351),
 ('with', 3291),
 ('have', 3193),
 ('as', 3122),
 ('not', 2980),
 ('com', 2942),
 ('or', 2832),
 ('they', 2712),
 ('was', 2651),
 ('at', 2590),
 ('but', 2589),
 ('if', 2562),
 ('subject', 2523),
 ('re', 2499),
 ('lines', 2445),
 ('organization', 2433),
 ('can', 2310),
 ('by', 2281),
 ('will', 2211),
 ('what', 1888),
 ('writes', 1885),
 ('would', 1857),
 ('an', 1812),
 ('he', 1791),
 ('article', 1777),
 ('one', 1772),
 ('about', 1729),
 ('my', 1674),
 ('all', 1638),
 ('there', 1612),
 ('do', 1556),
 ('has', 1533),
 ('we', 1503),
 ('so', 1465),
 ('space', 1445),
 ('your', 1431),
 ('no', 1385),
 ('more', 1382),
 ('some', 1344),
 ('posting', 1324),
 ('key', 1321),
 ('like', 1308),
 ('any', 1271),
 ('just', 1266),
 ('out', 1251),
 ('who', 

In [11]:
# Klasyfikacja tekstu

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word',ngram_range=(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [13]:
#wytrenowanie pipelinu
clf.fit(train_dataset.data, train_dataset.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [14]:
#proste testy wytrenowanego modelu
sentences = ['Asteroids are dangerous for Earth.', 
             'I am dreaming about new motorcycle.', 
             'I hate baseball games.', 
             'Did you see my C implementations?']

predicted_categories = clf.predict(sentences)

for sentence, predicted_category in zip(sentences, predicted_categories):
    print('%r => %s' % (sentence, train_dataset.target_names[predicted_category]))

'Asteroids are dangerous for Earth.' => sci.space
'I am dreaming about new motorcycle.' => rec.motorcycles
'I hate baseball games.' => rec.sport.baseball
'Did you see my C implementations?' => sci.crypt


In [15]:
#prosze wymyslec takie 4 zdania, ktore zostana zaklasyfikowane do kazdej z badanych klas:
#'alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian'

In [16]:
#testy wytrenowanego modelu
import numpy as np
predicted = clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target) 

0.9735015772870662

In [17]:
#macierz pomyłek
#'sci.crypt', 'rec.sport.baseball', 'rec.motorcycles', 'sci.space'
from sklearn.metrics import confusion_matrix
confusion_matrix(test_dataset.target, predicted, labels=[0,1,2,3])

array([[389,   0,   7,   2],
       [  1, 390,   3,   3],
       [  1,   4, 391,   0],
       [  1,   0,  20, 373]], dtype=int64)

In [18]:
#2. Prosze zmienic uzyty klasyfikator na drzewo decyzyjne i zbadac jakosc wynikow klasyfikacji

In [19]:
from sklearn import tree
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word',ngram_range=(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', tree.DecisionTreeClassifier()),
])

In [20]:
#wytrenowanie pipelinu
clf.fit(train_dataset.data, train_dataset.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier())])

In [21]:
#proste testy wytrenowanego modelu
sentences = ['Asteroids are dangerous for Earth.', 
             'I am dreaming about new motorcycle.', 
             'I hate baseball games.', 
             'Did you see my C implementations?']

predicted_categories = clf.predict(sentences)

for sentence, predicted_category in zip(sentences, predicted_categories):
    print('%r => %s' % (sentence, train_dataset.target_names[predicted_category]))

'Asteroids are dangerous for Earth.' => rec.sport.baseball
'I am dreaming about new motorcycle.' => rec.motorcycles
'I hate baseball games.' => rec.sport.baseball
'Did you see my C implementations?' => rec.sport.baseball


In [22]:
#testy wytrenowanego modelu
import numpy as np
predicted = clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target) 

0.8145110410094637

In [23]:
#macierz pomyłek
# 'sci.crypt', 'rec.sport.baseball', 'rec.motorcycles', 'sci.space'
confusion_matrix(test_dataset.target, predicted, labels=[0,1,2,3])

array([[340,  28,  18,  12],
       [ 21, 329,  17,  30],
       [ 21,  38, 323,  14],
       [ 24,  45,  26, 299]], dtype=int64)

In [24]:
#3. Prosze wytrenowac klasyfikator na zbiorze testujacym a przetestowac na zbiorze trenujacym

In [25]:
test_dataset = fetch_20newsgroups(
    subset='train', 
    categories=categories, 
    shuffle=True, 
    random_state=42)
train_dataset = fetch_20newsgroups(
    subset='test', 
    categories=categories, 
    shuffle=True, 
    random_state=42)
#klasy
print(train_dataset.target_names)
print(test_dataset.target_names)
#liczba wiadomosci
print(len(train_dataset.data))
print(len(test_dataset.data))

['rec.motorcycles', 'rec.sport.baseball', 'sci.crypt', 'sci.space']
['rec.motorcycles', 'rec.sport.baseball', 'sci.crypt', 'sci.space']
1585
2383


In [49]:
count_vect = CountVectorizer(analyzer='word', ngram_range=(1, 1))
X_train_counts = count_vect.fit_transform(train_dataset.data)
word_list = count_vect.get_feature_names_out()
count_list = X_train_counts.toarray().sum(axis=0)
word_count = dict(zip(word_list,count_list))
sorted(word_count.items(), key=lambda x: x[1], reverse=True)

[('the', 30530),
 ('to', 15205),
 ('of', 13501),
 ('and', 11909),
 ('in', 9861),
 ('is', 8380),
 ('that', 6952),
 ('it', 6349),
 ('for', 5829),
 ('on', 4651),
 ('from', 4554),
 ('you', 4510),
 ('edu', 4471),
 ('be', 4399),
 ('this', 4193),
 ('are', 3351),
 ('with', 3291),
 ('have', 3193),
 ('as', 3122),
 ('not', 2980),
 ('com', 2942),
 ('or', 2832),
 ('they', 2712),
 ('was', 2651),
 ('at', 2590),
 ('but', 2589),
 ('if', 2562),
 ('subject', 2523),
 ('re', 2499),
 ('lines', 2445),
 ('organization', 2433),
 ('can', 2310),
 ('by', 2281),
 ('will', 2211),
 ('what', 1888),
 ('writes', 1885),
 ('would', 1857),
 ('an', 1812),
 ('he', 1791),
 ('article', 1777),
 ('one', 1772),
 ('about', 1729),
 ('my', 1674),
 ('all', 1638),
 ('there', 1612),
 ('do', 1556),
 ('has', 1533),
 ('we', 1503),
 ('so', 1465),
 ('space', 1445),
 ('your', 1431),
 ('no', 1385),
 ('more', 1382),
 ('some', 1344),
 ('posting', 1324),
 ('key', 1321),
 ('like', 1308),
 ('any', 1271),
 ('just', 1266),
 ('out', 1251),
 ('who', 

In [27]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word',ngram_range=(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
#wytrenowanie pipelinu
clf.fit(train_dataset.data, train_dataset.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [28]:
#proste testy wytrenowanego modelu
sentences = ['Asteroids are dangerous for Earth.', 
             'I am dreaming about new motorcycle.', 
             'I hate baseball games.', 
             'Did you see my C implementations?']

predicted_categories = clf.predict(sentences)

for sentence, predicted_category in zip(sentences, predicted_categories):
    print('%r => %s' % (sentence, train_dataset.target_names[predicted_category]))

'Asteroids are dangerous for Earth.' => sci.space
'I am dreaming about new motorcycle.' => rec.motorcycles
'I hate baseball games.' => rec.sport.baseball
'Did you see my C implementations?' => rec.motorcycles


In [29]:
predicted = clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target) 

0.9697859840537139

In [30]:
# 'sci.crypt', 'rec.sport.baseball', 'rec.motorcycles', 'sci.space'
confusion_matrix(test_dataset.target, predicted, labels=[0,1,2,3])

array([[576,  10,   7,   5],
       [  5, 583,   8,   1],
       [  1,   1, 592,   1],
       [  2,  10,  21, 560]], dtype=int64)

In [31]:
#4. Prosze ustawic w odpowiednim miejscu parametr analyzer='char' i sprawdzic jak zmieniaja sie wyniki

In [32]:
train_dataset = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
test_dataset = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
#klasy
print(train_dataset.target_names)
print(test_dataset.target_names)
#liczba wiadomosci
print(len(train_dataset.data))
print(len(test_dataset.data))

['rec.motorcycles', 'rec.sport.baseball', 'sci.crypt', 'sci.space']
['rec.motorcycles', 'rec.sport.baseball', 'sci.crypt', 'sci.space']
2383
1585


In [50]:
count_vect = CountVectorizer(analyzer='char', ngram_range=(1, 1))
X_train_counts = count_vect.fit_transform(train_dataset.data)
word_list = count_vect.get_feature_names_out()
count_list = X_train_counts.toarray().sum(axis=0)
word_count = dict(zip(word_list,count_list))
sorted(word_count.items(), key=lambda x: x[1], reverse=True)

[(' ', 611986),
 ('e', 356576),
 ('t', 274214),
 ('a', 244374),
 ('o', 235500),
 ('i', 226347),
 ('n', 216152),
 ('s', 208562),
 ('r', 192674),
 ('h', 130791),
 ('l', 128842),
 ('c', 112438),
 ('d', 107127),
 ('u', 93488),
 ('m', 85388),
 ('p', 73989),
 ('.', 70497),
 ('g', 65579),
 ('y', 62716),
 ('f', 60465),
 ('-', 58052),
 ('b', 55879),
 ('w', 52034),
 ('\n', 48768),
 ('v', 33854),
 (',', 32937),
 ('k', 29792),
 (':', 23327),
 ('1', 21961),
 ('>', 21654),
 ('0', 20665),
 ("'", 12544),
 (')', 12342),
 ('=', 12049),
 ('2', 11888),
 ('(', 11469),
 ('j', 11011),
 ('3', 10130),
 ('9', 9656),
 ('@', 9244),
 ('_', 9067),
 ('*', 8902),
 ('x', 8593),
 ('5', 8571),
 ('4', 7558),
 ('"', 7244),
 ('z', 6870),
 ('6', 6142),
 ('8', 6060),
 ('7', 5630),
 ('?', 5533),
 ('|', 5082),
 ('/', 4533),
 ('q', 3984),
 ('!', 3021),
 ('<', 2891),
 (';', 1978),
 ('~', 1898),
 ('\t', 1635),
 ('#', 1520),
 ('^', 1369),
 (']', 1287),
 ('[', 1273),
 ('+', 1206),
 ('$', 1172),
 ('`', 847),
 ('&', 820),
 ('\\', 720

In [34]:
count_vect = CountVectorizer(analyzer='char', ngram_range=(5, 5))
X_train_counts = count_vect.fit_transform(train_dataset.data)
word_list = count_vect.get_feature_names()
count_list = X_train_counts.toarray().sum(axis=0)
word_count = dict(zip(word_list,count_list))
sorted(word_count.items(), key=lambda x: x[1], reverse=True)

[('-----', 31948),
 (' the ', 27356),
 (' and ', 10438),
 ('=====', 9864),
 ('ation', 7643),
 (' that', 6619),
 ('that ', 6043),
 ('_____', 5571),
 ('*****', 5337),
 (' for ', 5231),
 ('n the', 4746),
 ('tion ', 4451),
 (' of t', 4195),
 ('f the', 4010),
 ('t the', 3967),
 (' this', 3902),
 ('of th', 3848),
 ('this ', 3710),
 (' with', 3511),
 (' you ', 3492),
 ('e the', 3312),
 ('ould ', 3160),
 ('with ', 3113),
 ('ing t', 3107),
 ('ion: ', 3101),
 (' have', 3088),
 ('the s', 3061),
 ('tion:', 3039),
 ('have ', 3002),
 (' are ', 2982),
 (', and', 2818),
 ('in th', 2786),
 ('. the', 2738),
 ('bject', 2687),
 (' in t', 2674),
 ('e of ', 2664),
 ('organ', 2644),
 ('izati', 2589),
 ('zatio', 2589),
 ('rgani', 2584),
 ('.edu ', 2576),
 ('ganiz', 2559),
 ('subje', 2540),
 ('ubjec', 2540),
 (' they', 2532),
 ('from:', 2503),
 ('rom: ', 2501),
 ('nizat', 2491),
 ('aniza', 2488),
 ('lines', 2488),
 ('d the', 2487),
 ('nes: ', 2431),
 ('ines:', 2427),
 ('the c', 2426),
 (' not ', 2420),
 ('ject

In [51]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
#wytrenowanie pipelinu
clf.fit(train_dataset.data, train_dataset.target)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char')),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [52]:
#proste testy wytrenowanego modelu
sentences = ['Asteroids are dangerous for Earth.', 
             'I am dreaming about new motorcycle.', 
             'I hate baseball games.', 
             'Did you see my C implementations?']

predicted_categories = clf.predict(sentences)

for sentence, predicted_category in zip(sentences, predicted_categories):
    print('%r => %s' % (sentence, train_dataset.target_names[predicted_category]))

'Asteroids are dangerous for Earth.' => sci.crypt
'I am dreaming about new motorcycle.' => sci.crypt
'I hate baseball games.' => rec.sport.baseball
'Did you see my C implementations?' => sci.crypt


In [53]:
predicted = clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target) 

0.3747634069400631

In [54]:
# 'sci.crypt', 'rec.sport.baseball', 'rec.motorcycles', 'sci.space'
confusion_matrix(test_dataset.target, predicted, labels=[0,1,2,3])

array([[116,  90, 161,  31],
       [ 31, 119, 231,  16],
       [ 29,  32, 296,  39],
       [ 27,  56, 248,  63]], dtype=int64)

In [39]:
# zmieniamy ngram na 2,2

In [55]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='char',ngram_range=(5, 5))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
#wytrenowanie pipelinu
clf.fit(train_dataset.data, train_dataset.target)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', ngram_range=(5, 5))),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [56]:
#proste testy wytrenowanego modelu
sentences = ['Asteroids are dangerous for Earth.', 
             'I am dreaming about new motorcycle.', 
             'I hate baseball games.', 
             'Did you see my C implementations?']

predicted_categories = clf.predict(sentences)

for sentence, predicted_category in zip(sentences, predicted_categories):
    print('%r => %s' % (sentence, train_dataset.target_names[predicted_category]))

'Asteroids are dangerous for Earth.' => sci.space
'I am dreaming about new motorcycle.' => rec.motorcycles
'I hate baseball games.' => rec.sport.baseball
'Did you see my C implementations?' => sci.crypt


In [57]:
predicted = clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target) 

0.8845425867507887

In [58]:
# 'sci.crypt', 'rec.sport.baseball', 'rec.motorcycles', 'sci.space'
confusion_matrix(test_dataset.target, predicted, labels=[0,1,2,3])

array([[327,   4,  64,   3],
       [ 16, 344,  36,   1],
       [  6,   2, 385,   3],
       [  2,   1,  45, 346]], dtype=int64)

In [44]:
# zmieniamy ngram na 3,3

In [45]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word',ngram_range=(3, 3))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
#wytrenowanie pipelinu
clf.fit(train_dataset.data, train_dataset.target)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(3, 3))),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [46]:
#proste testy wytrenowanego modelu
sentences = ['Asteroids are dangerous for Earth.', 
             'I am dreaming about new motorcycle.', 
             'I hate baseball games.', 
             'Did you see my C implementations?']

predicted_categories = clf.predict(sentences)

for sentence, predicted_category in zip(sentences, predicted_categories):
    print('%r => %s' % (sentence, train_dataset.target_names[predicted_category]))

'Asteroids are dangerous for Earth.' => rec.motorcycles
'I am dreaming about new motorcycle.' => rec.motorcycles
'I hate baseball games.' => rec.motorcycles
'Did you see my C implementations?' => sci.crypt


In [47]:
predicted = clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target) 

0.9186119873817035

In [48]:
# 'sci.crypt', 'rec.sport.baseball', 'rec.motorcycles', 'sci.space'
confusion_matrix(test_dataset.target, predicted, labels=[0,1,2,3])

array([[371,  13,  12,   2],
       [ 17, 357,  10,  13],
       [ 11,  10, 364,  11],
       [  8,   9,  13, 364]], dtype=int64)